In [2]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.3 MB 6.2 MB/s eta 0:00:01
   --------------------- ------------------ 2.4/4.3 MB 6.2 MB/s eta 0:00:01
   ---------------------------------- ----- 3.7/4.3 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 6.1 MB/s  0:00:00




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data_utils  



In [5]:
import torch.nn as nn
mse_loss = nn.MSELoss()
predictions = torch.tensor([2.5, 0.0, 2.1])
targets = torch.tensor([3.0, -0.5, 2.0])
loss = mse_loss(predictions, targets)
# Equivalent to: ((2.5-3.0)² + (0.0-(-0.5))² + (2.1-2.0)²) / 3
# = (0.25 + 0.25 + 0.01) / 3 = 0.17
print(loss.item())  # Output: 0.17

0.17000000178813934


In [3]:
import pandas as pd

df = pd.read_csv("./data/customer_reviews.csv")
df.head(5)

,product_id,customer_id,rating,review_text,review_date,product_category,verified_purchase
0,P001,C1547,5,Absolutely love this product! Exceeded all my ...,2024-01-15,Electronics,True
1,P001,C2891,4,Great product overall. Works as advertised. On...,2024-01-18,Electronics,True
2,P002,C3456,1,Very disappointed. Stopped working after just ...,2024-01-20,Electronics,True
3,P002,C4123,2,Not worth the price. Quality feels cheap and i...,2024-01-22,Electronics,False
4,P003,C5678,5,Perfect! Exactly what I needed. The design is ...,2024-01-25,Home & Kitchen,True


In [8]:
# Create sentiment categories based on numeric rating
df['sentiment_category'] = pd.cut(
    df['rating'],
    bins=[0, 2, 3, 5],
    labels=['Negative', 'Neutral', 'Positive']
)

print("-" * 50)
print(df)

# Group by category and sentiment, aggregating key metrics
analysis = (
    df.groupby(['product_category', 'sentiment_category'])
      .agg(
          rating_count=('rating', 'count'),
          product_id_nunique=('product_id', 'nunique')
      )
      .reset_index()
)

print("-" * 50)
print(analysis)

# Fill NaNs resulting from empty groups with zeros
analysis = analysis.fillna({
    'rating_count': 0,
    'product_id_nunique': 0
})

# Calculate percentage of negative reviews safely
category_counts = df.groupby('product_category').size()
negative_counts = (
    df[df['sentiment_category'] == 'Negative']
    .groupby('product_category')
    .size()
)
negative_pct = (negative_counts / category_counts * 100).fillna(0)

# Display results
print("Review analysis by category and sentiment:")
print(analysis)
print("\nPercentage of negative reviews by category:")
print(negative_pct.round(1))

--------------------------------------------------
   product_id customer_id  rating  \
0        P001       C1547       5   
1        P001       C2891       4   
2        P002       C3456       1   
3        P002       C4123       2   
4        P003       C5678       5   
5        P003       C6234       5   
6        P004       C7890       3   
7        P005       C8345       4   
8        P005       C9012       5   
9        P006       C1678       1   
10       P006       C2345       2   
11       P007       C3901       5   
12       P007       C4567       4   
13       P008       C5234       3   
14       P008       C6890       4   
15       P009       C7456       5   
16       P009       C8123       5   
17       P010       C9678       2   
18       P010       C1234       1   
19       P011       C2567       5   
20       P011       C3890       4   
21       P012       C4456       3   
22       P012       C5123       4   
23       P013       C6789       5   
24       P013       C723

C:\Users\chars1\AppData\Local\Temp\ipykernel_4272\3092358929.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['product_category', 'sentiment_category'])


In [10]:
# --------------------------------------------------------------
# Cell 1: Extract the reviews into a list called combined_reviews
# --------------------------------------------------------------

# df is already loaded in a previous cell (customer_reviews.csv)
combined_reviews = df['review_text'].tolist()

# Quick sanity‑check (optional)
print(f"Extracted {len(combined_reviews)} reviews.")

Extracted 51 reviews.


In [12]:
# --------------------------------------------------------------
# Cell 2: Summarize the extracted reviews using Ollama
# ----------------------------------------------------------------------------

import ollama  # <-- Ollama client
import os

# (No API key is needed for a local Ollama server.)

# Compose the prompt, inserting the list of reviews
prompt = f"""
You are analyzing customer reviews to identify product and service issues.

Below are customer reviews for a set of products. Please analyze them and provide:

1. **Common Product Issues**: List the main problems customers reported about the product itself (quality, functionality, features, etc.)
2. **Common Service Issues**: List the main problems customers reported about the service (shipping, customer support, packaging, etc.)
3. **Frequency**: For each issue, estimate how often it appears (e.g., "mentioned frequently", "occasional complaint", "rare issue")

Format your response clearly with headers and bullet points.

CUSTOMER REVIEWS:
{combined_reviews}
"""

# Call the Ollama model (pick any model you have – e.g. “llama3.2”)
response = ollama.chat(
    model="gpt-oss:20b",          # change to your available model name
    messages=[{"role":"user","content":prompt}],
    #temperature=0.3,           # keep the answer focused
    stream=False
)

# The Ollama client returns a dict with a 'message' key
summary = response["message"]["content"]
print(summary)
# import requests
# url = "http://localhost:11434/api/chat"
# data = {
#     "model":"llama3.2",
#     "messages":[{"role":"user","content":prompt}],
#     "temperature":0.3,
#     "stream":False
# }
# r = requests.post(url, json=data)
# summary = r.json()["message"]["content"]

**Common Product Issues**

| Issue | Typical Complaint | Frequency |
|-------|-------------------|-----------|
| **Poor build quality / cheap materials** | “Terrible quality. Broke on first use.” “Cheaply made. Not what I expected based on the product photos.” | Frequently |
| **Breaks / fails quickly** | “Stopped working after just 2 weeks.” “Stopped functioning after a week.” “Broke on first use.” | Frequently |
| **Misleading description / photos** | “Not as described.” “Product looks nothing like the photos.” “Product description was very misleading.” | Frequently |
| **Under‑performance / not as advertised** | “Quality feels cheap and it doesn’t perform as well as described.” “Product does not work as advertised.” | Frequently |
| **Limited or missing features** | “Does the job but nothing special.” “The actual product looks nothing like the photos.” | Occasional |
| **Design / usability flaws** | “Minor design flaw.” “Very disappointed.” | Occasional |
| **Limited color / style o

In [13]:
# --------------------------------------------------------------
# Cell 3: Print the summary and save it as customer_reviews.md
# --------------------------------------------------------------

print("=== Customer‑Review Summary ===\n")
print(summary)

# Write the summary to a markdown file
with open("customer_reviews.md", "w", encoding="utf-8") as md_file:
    md_file.write(summary)

print("\nSummary written to customer_reviews.md")

=== Customer‑Review Summary ===

**Common Product Issues**

| Issue | Typical Complaint | Frequency |
|-------|-------------------|-----------|
| **Poor build quality / cheap materials** | “Terrible quality. Broke on first use.” “Cheaply made. Not what I expected based on the product photos.” | Frequently |
| **Breaks / fails quickly** | “Stopped working after just 2 weeks.” “Stopped functioning after a week.” “Broke on first use.” | Frequently |
| **Misleading description / photos** | “Not as described.” “Product looks nothing like the photos.” “Product description was very misleading.” | Frequently |
| **Under‑performance / not as advertised** | “Quality feels cheap and it doesn’t perform as well as described.” “Product does not work as advertised.” | Frequently |
| **Limited or missing features** | “Does the job but nothing special.” “The actual product looks nothing like the photos.” | Occasional |
| **Design / usability flaws** | “Minor design flaw.” “Very disappointed.” | Occasio